# Hallucination removal techniques from LLMs

## In this notebook we focus on the how to remove hallucination from an LLM

##Cross encoder reranking

In [1]:
from sentence_transformers import CrossEncoder

# 1. Load a pre-trained Cross-Encoder model
# Example: 'cross-encoder/ms-marco-MiniLM-L-6-v2'
model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# 2. Define your query and candidate documents
query = "What is Cross-Encoder used for?"
candidates = [
    "Cross-Encoders are used for reranking search results.",
    "Bi-Encoders encode queries and documents separately.",
    "RAG pipelines can use Cross-Encoders for precise scoring."
]

# 3. Prepare input pairs (query, document)
pairs = [(query, doc) for doc in candidates]

# 4. Compute relevance scores
scores = model.predict(pairs)

# 5. Rerank documents by score (descending)
reranked = [doc for _, doc in sorted(zip(scores, candidates), reverse=True)]

# 6. Output
print("Reranked documents:")
for doc in reranked:
    print("-", doc)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Reranked documents:
- Cross-Encoders are used for reranking search results.
- RAG pipelines can use Cross-Encoders for precise scoring.
- Bi-Encoders encode queries and documents separately.


##Dense Passage Retrieval (DPR)

In [7]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer, DPRContextEncoder, DPRContextEncoderTokenizer
import torch
import torch.nn.functional as F

# 1. Load DPR encoders and tokenizers
query_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
query_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

# 2. Example query and passages
query = "What is Dense Passage Retrieval?"
passages = [
    "Dense Passage Retrieval is a bi-encoder based retrieval method.",
    "Cross-Encoders are used for reranking candidate documents.",
    "DPR encodes queries and passages into dense vectors for fast search."
]

# 3. Encode query
query_inputs = query_tokenizer(query, return_tensors="pt")
query_vec = query_encoder(**query_inputs).pooler_output  # [1, hidden_size]

# 4. Encode passages
context_inputs = context_tokenizer(
    passages,
    padding=True,
    truncation=True,
    max_length=256,  # truncate long passages
    return_tensors="pt"
)

#context_inputs = context_tokenizer(passages, padding=True, truncation=True, return_tensors="pt")
context_vecs = context_encoder(**context_inputs).pooler_output  # [num_passages, hidden_size]

# 5. Compute cosine similarity
scores = F.cosine_similarity(query_vec, context_vecs)
top_k = 2
top_results = torch.topk(scores, top_k)

print("Top passages:")
for score, idx in zip(top_results.values, top_results.indices):
    print(f"{passages[idx]} (score: {score.item():.4f})")


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the

Top passages:
Dense Passage Retrieval is a bi-encoder based retrieval method. (score: 0.7644)
DPR encodes queries and passages into dense vectors for fast search. (score: 0.6054)


## RAFT --> Retrival Augmented FineTuning

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.optim import AdamW

# ======== 1. Setup ========
model_name = "gpt2"  # pretrained LLM
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

optimizer = AdamW(model.parameters(), lr=5e-5)

# Tiny toy dataset
training_data = [
    {
        "query": "What is RAFT?",
        "target_answer": "RAFT fine-tunes LLMs using retrieved documents.",
        "retrieved_docs": ["RAFT improves LLM accuracy by leveraging retrieved knowledge."]
    },
    {
        "query": "What is DPR?",
        "target_answer": "DPR is a dense retriever for passages.",
        "retrieved_docs": ["DPR encodes queries and passages into dense vectors."]
    }
]

# ======== 2. RAFT Fine-Tuning Loop ========
model.train()  # set model to training mode

for example in training_data:
    query = example["query"]
    target_answer = example["target_answer"]
    retrieved_docs = example["retrieved_docs"]

    # 2a. Combine query + retrieved docs + target answer
    context_text = f"Query: {query}\nContext: {' '.join(retrieved_docs)}\nAnswer: "
    full_text = context_text + target_answer

    # 2b. Tokenize full input
    inputs = tokenizer(full_text, return_tensors="pt")

    # 2c. Create labels
    labels = inputs.input_ids.clone()
    # Mask out query + context so loss is only computed on target answer
    answer_start = len(tokenizer(context_text).input_ids)
    labels[0, :answer_start] = -100  # -100 means ignore

    # 2d. Forward pass → RAFT fine-tuning happens here
    outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    print(f"Training loss: {loss.item():.4f}")

    # 2e. Backpropagation → update weights
    loss.backward()          # <-- RAFT fine-tuning step
    optimizer.step()         # <-- RAFT fine-tuning step
    optimizer.zero_grad()    # reset gradients for next example

# ======== 3. Inference ========
model.eval()  # set model to eval mode
query = "Explain RAFT."
retrieved_docs = ["RAFT fine-tunes LLMs using retrieved documents."]
context_text = f"Query: {query}\nContext: {' '.join(retrieved_docs)}\nAnswer: "
inputs = tokenizer(context_text, return_tensors="pt")

outputs = model.generate(**inputs, max_length=50)
print("Generated answer:", tokenizer.decode(outputs[0], skip_special_tokens=True))


Training loss: 4.6769
Training loss: 4.6720


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated answer: Query: Explain RAFT.
Context: RAFT fine-tunes LLMs using retrieved documents.
Answer:  The RAFT document is a document that is retrieved from the RAFT database.  The document is a document that
